## Assignment 2 (50 marks)
#### =====================================================================================================
### Deadline: 09/28 11:59 pm
#### =====================================================================================================

### Problem 1: Perceptron Learning (15 marks)

The dataset `lab02_dataset_1.csv` has a *3-dimensional input space* and a class label of *Positive* and *Negative*. For this task, you are **not allowed** to use any *functionalities* of the `sklearn` module.

### 1.a (10 marks)

Write a function `my_perceptron()` which applies the perceptron algorithm (refer to the lecture slide covering linear separators for details of this algorithm) on the dataset to create a linear separator. `my_perceptron()` takes the dataset as its input and returns a ***3-dimensional weight vector*** which can be used to create the linear separator (assume `bias = 0`). Use the *initial weights* `w = [3.5,0.5,-2.5]`. Use a classification threshold of `99%` i.e., `my_perceptron()` will terminate once the misclassification rate is less than `1%`.

In [ ]:
import pandas as pd
import numpy as np

def my_perceptron():
    data = pd.read_csv('lab02_dataset_1.csv')

    inputs = data[['X', 'Y', 'Z']].to_numpy()
    classes = data['Class'].map({'Negative': -1, 'Positive': 1}).to_numpy()

    weights = np.array([3.5, 0.5, -2.5])
    bias = 0
    
    for iter in range(1000): 
        err = 0
        for i in range(len(inputs)):
            a = np.dot(weights, inputs[i]) + bias
            predication = 1 if a >= 0 else -1
            if predication != classes[i]:
                weights += classes[i] * inputs[i]
                bias += classes[i]
                err += 1
        err_rate = err / len(data)
        if err_rate <= 0.01:
            break
    
    return weights

if __name__ == "__main__":
    print(my_perceptron())

[0.14536582 2.30319913 0.34978878]


### 1.b (5 marks)

Create a *3D plot* which showcases the dataset in a 3D-space alongwith the *linear separator* you obtained from `my_perceptron()`. Use two different colors to represent the data points belonging in the two classes for ease of viewing.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

### Problem 2: SVM Classification (35 marks)

`lab02_dataset_2.xlsx` contains the claim history of 27,513 homeowner policies. The following table describes the eleven columns in the dataset.

| Name | Description |
| --- | --- |
| policy | Policy Identifier |
| exposure | Duration a Policy exposed in a Year |
| num_claims | Number of Claims in a Year |
| amt_claims | Total Claim Amount in a Year	|
| f_primary_age_tier | Age Tier of Primary Insured |
| f_primary_gender | Gender of Primary Insured |
| f_marital | Marital Status of Primary Insured |
| f_residence_location | Location of Residence Property |
| f_fire_alarm_type | Fire Alarm Type |
| f_mile_fire_station | Distance to Nearest Fire Station |
| f_aoi_tier | Amount of Insurance Tier |

We want to predict the *Frequency* which is the *number of claims per unit of exposure* using the above features.  We first divide the reported number of claims by the exposure. This gives us the *Frequency*.  Next, we put the policies into five groups according to their *Frequency* values. We will use this *Group* as our target variable which has five classes.

| Group | Values |
| :--- | :--- |
| 0 | Frequency = 0 |
| 1 | 0 < Frequency <= 1 |
| 2 | 1 < Frequency <= 2 |
| 3 | 2 < Frequency <= 3 |
| 4 | 3 < Frequency |

### 2.a (6 marks)
Create a new column for the dataset which will indiciate the *Frequency Group* and output the updated dataset.

### 2.b (6 marks)
There are seven categorial features in the dataset namely, *f_aoi_tier, f_primary_age_tier, f_fire_alarm_type, f_marital, f_mile_fire_station, f_primary_gender, f_residence_location*. Display all the unique values of these seven categories.

### 2.c (6 marks)
We will train SVM models using those seven categorical features. However, their values are currently all categorical data, but SVM requires them to be numerical. Perform `One-hot Encoding` on these features to obtain an updated dataset which has only numerical values.

### 2.d (6 marks)
Divide the observations into training and testing partitions. Observations whose *Policy Identifier* starts with the letters A, G, and P will go to the training partition. The remaining observations go to the testing partition. Output the total number of policies present in the Training partition and Testing partition.

### 2.e (6 marks)
Train an SVM model using [`LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). The input features will be the encoded version of the feature set *f_aoi_tier, f_primary_age_tier, f_fire_alarm_type, f_marital, f_mile_fire_station, f_primary_gender, f_residence_location* and the output is the *Frequency Group*. Use `verbose=1` to observe the optimization steps during the training process.

### 2.f (5 marks)
Compute and output the accuracy score on the testing partition.